## 匯入套件 & 讀取資料

In [150]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import requests
import json
import csv
import numpy as np
import pandas as pd


total_df = pd.read_csv('../data/preprocess/activity.csv')

activity_df = total_df.loc[total_df.index <= 5]

activity_df

,id,title,subTitle,content,fee,holders,objectives,branches,tags
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,NaN,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'id': 7, 'text': '化學', 'type': 'area', 'tren..."
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,NaN,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'id': 6, 'text': '運動', 'type': 'area', 'tren..."
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,NaN,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'id': 9, 'text': '心理', 'type': 'area', 'tren..."
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,NaN,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[]
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,NaN,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'id': 13, 'text': '政治', 'type': 'area', 'tre..."
5,2668c627-d2d2-4d24-4d5b-08db52fbe02a,2023中正大學勞工營,NaN,中正大學是全台唯一擁有勞工系的國立大學。為了讓更多人知道勞工系，也更加了解這個系所學，並且能...,[],[],[],"[{'id': 136, 'branchName': 'General', 'locatio...",[]


## 訓練 CKIP 模型

In [151]:
from ckip_transformers import __version__
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker


# Show version
print(__version__)

# Initialize drivers
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="albert-base")
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="albert-base")
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="albert-base")
print("Initializing drivers ... all done")
print()

0.3.4
Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... all done



## 清除沒必要的單詞

In [152]:
# clean function
def clean(sentence_ws, sentence_pos):
  short_with_pos = []
  short_sentence = []
  stop_pos = set(['Nep', 'Nh', 'Nb', 'Neu']) # 這 3 種詞性不保留 指代定詞 代名詞 專有名詞 數字
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉名詞裡的某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 只剩一個字的詞也不留
    is_not_one_charactor = not (len(word_ws) == 1)
    
    # 組成串列
    if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")
  return (" ".join(short_sentence), " ".join(short_with_pos))

## 執行斷詞

In [188]:
def CKIP_segment(content):
    ws = ws_driver([content])
    pos = pos_driver(ws)
    
    seg = ""
    for sentence_ws, sentence_pos in zip(ws, pos):
        (short, res) = clean(sentence_ws, sentence_pos)
        seg += short
    
    return seg

activity_df['seg'] = activity_df['content'].apply(CKIP_segment)

activity_df

Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


,id,title,subTitle,content,fee,holders,objectives,branches,tags,seg
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,NaN,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'id': 7, 'text': '化學', 'type': 'area', 'tren...",檢定 分級 級別 檢定 範圍 占分 部分 高一 高三 範圍 為主 部分 大學 範圍 為主 中...
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,NaN,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'id': 6, 'text': '運動', 'type': 'area', 'tren...",陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗...
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,NaN,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'id': 9, 'text': '心理', 'type': 'area', 'tren...",專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 ...
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,NaN,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[],課程 期別 人才 研修 初階班 週六 週日 上課 人數 額滿 為止 適用 對象 高中 高職 ...
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,NaN,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'id': 13, 'text': '政治', 'type': 'area', 'tre...",何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸...
5,2668c627-d2d2-4d24-4d5b-08db52fbe02a,2023中正大學勞工營,NaN,中正大學是全台唯一擁有勞工系的國立大學。為了讓更多人知道勞工系，也更加了解這個系所學，並且能...,[],[],[],"[{'id': 136, 'branchName': 'General', 'locatio...",[],大學 擁有 勞工系 大學 更多 知道 勞工系 了解 高中生 大學 科系 選擇 生涯 規劃 提...


## 存入資料

In [191]:
activity_df.to_csv('../data/preprocess/activity.csv', index=False)

activity_df = pd.read_csv('../data/preprocess/activity.csv')

activity_df

,id,title,subTitle,content,fee,holders,objectives,branches,tags,seg
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,NaN,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'id': 7, 'text': '化學', 'type': 'area', 'tren...",檢定 分級 級別 檢定 範圍 占分 部分 高一 高三 範圍 為主 部分 大學 範圍 為主 中...
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,NaN,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'id': 6, 'text': '運動', 'type': 'area', 'tren...",陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗...
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,NaN,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'id': 9, 'text': '心理', 'type': 'area', 'tren...",專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 ...
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,NaN,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[],課程 期別 人才 研修 初階班 週六 週日 上課 人數 額滿 為止 適用 對象 高中 高職 ...
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,NaN,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'id': 13, 'text': '政治', 'type': 'area', 'tre...",何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸...
5,2668c627-d2d2-4d24-4d5b-08db52fbe02a,2023中正大學勞工營,NaN,中正大學是全台唯一擁有勞工系的國立大學。為了讓更多人知道勞工系，也更加了解這個系所學，並且能...,[],[],[],"[{'id': 136, 'branchName': 'General', 'locatio...",[],大學 擁有 勞工系 大學 更多 知道 勞工系 了解 高中生 大學 科系 選擇 生涯 規劃 提...


## 設立使用者資料

In [222]:
# 假設使用者活動紀錄
userli = [[1, 2], [1, 2, 4], [3]]

user_seg = [' '.join(activity_df.loc[userli[1], 'seg'].values.tolist())]

user_seg

['陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗 實務 操作 學習 帆船 相關 知識 培養 隊員 探索 解決 問題 團隊 合作 無限 潛能 駕馭 風帆 享受 勇敢 航向 大海 學習 目標 認識 帆船 運動 操作 認識 帆船 部件 知道 方位 知道 帆船 術語 實作 帆船 結繩 掌舵 讓船 航行 團隊 分工 共作 學習 目標 操作 重型 帆船 發展 團隊 合作 關係 操作 帆船 部件 理解 方位 熟悉 帆船 術語 實作 帆船 結繩 操作 帆讓船 航行 團隊 分工合作 活動 場域 西子灣 哨船頭 遊艇 碼頭 安平港 安平 漁港 遊艇 碼頭 興達港 興達港 情人 碼頭 遊艇 碼頭 專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 大學 社會 科學院 三樓 演講廳 三樓 電腦 教室 授課 講師 成功 大學 心理學系 教授 成功 大學 心智 影像 研究 中心 大學 認知 科學 博士 普林斯頓 大學 神經 科學 研究院 博士 研究 研究 興趣 臉孔 物體 辨識 決策 神經 科學 開放 神經 科學 開設 相關 課程 認知 神經 科學 電影 功能性 磁振 造影 理論 實務 功能性 磁振 造影 進階 分析 何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸 遊戲 營期 真實 總統 國會 公投案 選舉 精要 濃縮版 政策 公投案 擬定 遊說 政治 學長姐 扮演 NPC 政見 發表 造勢 跨隊 結盟 手段 模擬 政壇 生態 目標 獲得 選舉 勝利 選擇 角色 大政 奉還 小隊 配置 分為 政黨 小隊 利益 團體 屬於 捍衛 信念 推動 公投案 利團 取得 執政權 改變 政治 現狀 政黨 這裡 發揮 影響力 憑藉 謀略 膽識 獲得 一席之地 豐富 營隊 體驗 激烈 模擬 選戰 參與 臺灣 政治 文化 常見 綁樁 結盟 政論 節目 造勢 晚會 精彩 活動 政黨 利團 發布 媒體 新聞 宣傳 理念 官方 媒體 播報 即時 新聞 擔任 公正 角色 地方 勢力 富商 權貴 勞動 階級 小隊員 倚靠 謹惕 對象 選舉 走向 具有 影響力 課程 安排 學術 顧問 小隊 主題 專題 演講 邀集 臺灣 政壇 重量級 講者 學員 進行 主題式 演講 隊員 課程 議題 事務 獲得

## 算出使用者和活動的 TF-IDF 特徵矩陣，並算出相似值

In [228]:
vectorizer = TfidfVectorizer()

# 目的是學習每個單詞在所有文本中的重要性，轉換的目的是將每個文本轉換成一個數值向量
activity_vec = vectorizer.fit_transform(activity_df['seg'].values)

# 已經有一個訓練好的向量化模型，並且想要將新的文本資料轉換成與這個模型相同的向量形式
user_vec = vectorizer.transform(user_seg)

# 計算使用者向量和語料庫中所有活動向量之間的餘弦相似度
user_activity_matrix = cosine_similarity(user_vec, activity_vec)

# 透過增加字元的方式調整單詞權重

user_activity_matrix = pd.DataFrame(user_activity_matrix, index=[1], columns=activity_df.index)

user_activity_matrix

,0,1,2,3,4,5
1,0.004826,0.60165,0.426654,0.073386,0.700897,0.139476


## 找出最相似的活動

In [227]:
def get_the_most_similar_activitys(user_id, user_movie_matrix, num):
    """Find the top-n movies most similar to the user"""
    user_vec = user_movie_matrix.loc[user_id].values 
    sorted_index = np.argsort(user_vec)[::-1][:num]
    return list(user_movie_matrix.columns[sorted_index])

activity_ids = get_the_most_similar_activitys(0, user_activity_matrix, 3)

# print(activity_df[activity_df.index.isin(activity_ids)]["title"])
# print(activity_df[activity_df.index.isin(activity_ids)]['activity_id'])


# activity_df[activity_df.index.isin(activity_ids)]
activity_ids


[4, 1, 2]

In [149]:
import pandas as pd
import numpy as np

# 創建一個 5x3 的隨機數據矩陣
data = np.random.randn(5, 3)

# 將隨機數據矩陣轉換為 DataFrame
df = pd.DataFrame(data, columns=['A', 'B', 'C'])

# 輸出 DataFrame
df.loc[[1, 2, 4], 'B']

1   -0.611840
2    1.356513
4   -0.630494
Name: B, dtype: float64